# Performance Benchmark

This notebook benchmarks **StringCompare** against four other string comparison libraries, namely [jellyfish](https://github.com/jamesturk/jellyfish), [py_stringmatching](https://github.com/anhaidgroup/py_stringmatching), [textdistance](https://github.com/life4/textdistance), and [python-levenshtein](https://pypi.org/project/python-Levenshtein/).

We evaluate the performance of string metrics when doing the 250,000 all-to-all comparisons among a list of 500 business names taken from the [Registre des entreprises du Québec](https://www.registreentreprises.gouv.qc.ca).

Overall python-levenshtein is the fastest with its efficient CPython implementation. However the library has a more limited scope. **StringCompare** comes second in terms of speed.

## Benchmark

First let's load required packages and the business names dataset.

In [137]:
import pandas as pd
import numpy as np
import timeit

import stringcompare
import jellyfish
import py_stringmatching
import textdistance
import Levenshtein

names = pd.read_csv("data/REQ-names.csv", header=None)[0].values

Next we define the string metrics under consideration and put them in a tidy dataframe.

In [138]:
functions = {
    'stringcompare': {
        'Levenshtein': stringcompare.Levenshtein(),
        'Damerau-Levenshtein': stringcompare.DamerauLevenshtein(),
        'Jaro': stringcompare.Jaro(),
        'Jaro-Winkler': stringcompare.JaroWinkler(),
        #'LCSDistance': stringcompare.LCSDistance(),
        #'CharacterDifference': stringcompare.CharacterDifference(),
    },
    'jellyfish': {
        'Levenshtein': jellyfish.levenshtein_distance,
        'Damerau-Levenshtein': jellyfish.damerau_levenshtein_distance,
        'Jaro': jellyfish.jaro_distance,
        'Jaro-Winkler': jellyfish.jaro_winkler,
    },
    'py_stringmatching': {
        'Levenshtein': py_stringmatching.Levenshtein().get_raw_score,
        'Jaro': py_stringmatching.Jaro().get_raw_score,
        'Jaro-Winkler': py_stringmatching.Jaro().get_raw_score,
    },
    'Levenshtein': {
        'Levenshtein': Levenshtein.distance,
        'Jaro': Levenshtein.jaro,
        'Jaro-Winkler': Levenshtein.jaro_winkler,
    },
    'textdistance': {
        'Levenshtein': textdistance.levenshtein,
        'Damerau-Levenshtein': textdistance.damerau_levenshtein,
        'Jaro': textdistance.jaro,
        'Jaro-Winkler': textdistance.jaro_winkler,
    }
}

data = (
    pd.DataFrame.from_dict(functions)
    .melt(ignore_index=False, var_name="package", value_name="callable")
    .reset_index()
    .rename(columns={'index':'function'})
)

Finally, we time execution for pairwise comparisons in the `names` dataset.

In [139]:
def time(function):
    if pd.isna(function):
        return pd.NA
    return timeit.timeit(lambda: [function(s, t) for s in names for t in names], number=1)

data = data.assign(time=lambda data: [time(x.callable) for _, x in data.iterrows()])

Results are shown below (times are in seconds):

In [150]:
(
    data
    .drop(columns="callable")
    .pivot(index="function", columns="package", values="time")
    .style
    .highlight_min(axis=1)
    .set_caption("Run time (seconds) for 250,000 comparisons")
)

package,Levenshtein,jellyfish,py_stringmatching,stringcompare,textdistance
function,,,,,
Damerau-Levenshtein,,1.003565,,0.716120,1.830349
Jaro,0.137128,0.478374,1.454673,0.418118,1.090796
Jaro-Winkler,0.139616,0.506712,1.471088,0.423561,1.155257
Levenshtein,0.286207,0.461220,8.039414,0.401387,1.061479
